In [ ]:
import os
import urllib.request
import mxnet as mx

# Download pre-trained Resnet-18 model to directory
# For downloading a file to local disk
def download(url,target_dir):
    filename = url.split('/')[-1]
    if not os.path.exists(os.path.join(target_dir,filename)):
        urllib.request.urlretrieve(url,os.path.join(target_dir,filename))
        
download('http://data.dmlc.ml/mxnet/models/imagenet/resnet/18-layers/resnet-18-0000.params','./model')
download('http://data.dmlc.ml/mxnet/models/imagenet/resnet/18-layers/resnet-18-symbol.json','./model')        

In [ ]:
import mxnet as mx

# Load the pre-trained model
sym, arg_params, aux_params = mx.model.load_checkpoint('./model/resnet-18', 0)

# Get Data Iterators  image data and labels.
def get_iterators(batch_size, data_shape=(3, 224, 224)):
    train = mx.io.ImageRecordIter(
        path_imgrec         = './data/cars_train.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        shuffle             = True,
        rand_crop           = True,
        rand_mirror         = True)
    val = mx.io.ImageRecordIter(
        path_imgrec         = './data/cars_val.rec',
        data_name           = 'data',
        label_name          = 'softmax_label',
        batch_size          = batch_size,
        data_shape          = data_shape,
        rand_crop           = False,
        rand_mirror         = False)
    return (train, val)

In [ ]:
# Remove the ImageNet 1000 layer classification layer and attach 196 dimensional cars layer
def get_fine_tune_model(symbol, arg_params, num_classes, layer_name='flatten0'):
    """
    symbol: the pretrained network symbol
    arg_params: the argument parameters of the pretrained model
    num_classes: the number of classes for the fine-tune datasets
    layer_name: the layer name before the last fully-connected layer
    """
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output'] # remove the original fully connected layer
    net = mx.symbol.FullyConnected(data=net, num_hidden=num_classes, name='fc1')
    net = mx.symbol.SoftmaxOutput(data=net, name='softmax')
    mx.viz.plot_network(net) # Visualize the network
    new_args = dict({k:arg_params[k] for k in arg_params if 'fc1' not in k})
    return (net, new_args)

In [ ]:
# Train the model
import logging
head = '%(asctime)-15s %(message)s'
logging.basicConfig(level=logging.DEBUG, format=head)

def fit(symbol, arg_params, aux_params, train, val, batch_size, num_gpus, num_epoch):
    devs = [mx.gpu(i) for i in range(num_gpus)]
    mod = mx.mod.Module(symbol=symbol, context=devs)
    mod.fit(train, val,
        num_epoch=num_epoch, # Number of epochs
        arg_params=arg_params,
        aux_params=aux_params,
        allow_missing=True,
        batch_end_callback = mx.callback.Speedometer(batch_size, 10),
        kvstore='device',
        optimizer='sgd',
            #adam or sgd
        optimizer_params={'learning_rate':0.02}, # Learning rate
        initializer=mx.init.Xavier(rnd_type='gaussian', factor_type="in", magnitude=2),
        eval_metric='acc',
        epoch_end_callback = mx.callback.do_checkpoint('./model/cars-finetuned'))
    metric = mx.metric.Accuracy()
    return mod.score(val, metric)

num_classes = 196 # Number of categories in cars dataset
batch_per_gpu = 200 # Batch size
num_gpus = 1 # Since we only have a single GPU
num_epoch = 500 # Number of epochs that we are going to train for

(new_sym, new_args) = get_fine_tune_model(sym, arg_params, num_classes)

batch_size = batch_per_gpu * num_gpus
(train, val) = get_iterators(batch_size)
mod_score = fit(new_sym, new_args, aux_params, train, val, batch_size, num_gpus, num_epoch)

In [ ]:
# Load the finetuned model for prediction on new images
import scipy.io # if this fails, install scipy 'conda install scipy'
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
import mxnet as mx
 
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

num_epoch = 239

sym, arg_params, aux_params = mx.model.load_checkpoint('./model/cars-finetuned', num_epoch)
mod = mx.mod.Module(symbol=sym, context=mx.gpu(), label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

# Load the label file for getting car name
label_file = scipy.io.loadmat('./data/cars_annos.mat')


def get_image(url, show=False):
    # download and show the image
    fname = mx.test_utils.download(url)
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url,label_file):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    object_categories = [i[0] for i in label_file['class_names'][0]]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], object_categories[i]))

# Get prediction on an image

test_url = 'http://ai.stanford.edu/~jkrause/cars/car5.jpg'
#test_url = ''
predict(test_url,label_file)
